In [2]:
import duckdb

### Enable Advance Profiling

In [6]:
duckdb.execute(
    """
        -- PRAGMA disable_profiling
        PRAGMA enable_profiling = 'query_tree';
        -- PRAGMA profiling_output = './profiling.txt';
    """
)

## Display the profiling out
# duckdb.sql("SELECT current_setting('profiling_output')")

# 3. DuckDB with Python API 

## 3.2 Import Data

### Import Data from CSV

In [3]:
duck_df = duckdb.read_csv("data/athens_data.csv")
type(duck_df)

duckdb.duckdb.DuckDBPyRelation

## 3.3 Basic Query Syntax


### Transform Data

#### SQL Expression

In [12]:
# %%timeit
duckdb.sql(
    """
        SELECT 
            Date AS date,
            station_name, 
            Temp AS temp, 
            "PM2.5" AS pm2_5,
        FROM duck_df 
        WHERE 
            Temp BETWEEN 10 AND 12
            AND Date BETWEEN DATE '2021-01-01' AND DATE '2021-01-31'
        ORDER BY Temp
        LIMIT 5
    """
)

┌─────────────────────┬──────────────────────────────┬────────────────────┬───────────────────┐
│        date         │         station_name         │        temp        │       pm2_5       │
│      timestamp      │           varchar            │       double       │      double       │
├─────────────────────┼──────────────────────────────┼────────────────────┼───────────────────┤
│ 2021-01-08 00:00:00 │ PANACEA_001                  │ 10.000026105949011 │              34.0 │
│ 2021-01-25 23:00:00 │ PANACEA_000                  │ 10.000708271896553 │            8.1875 │
│ 2021-01-26 21:00:00 │ Athens CAMS station 1        │ 10.000905261684789 │         2.1124547 │
│ 2021-01-25 23:00:00 │ CleanAir in Greece - Thiseio │ 10.000995675604765 │ 8.657692307692308 │
│ 2021-01-29 19:00:00 │ PARASKEVI                    │ 10.001125804259836 │               8.0 │
└─────────────────────┴──────────────────────────────┴────────────────────┴───────────────────┘

#### Method Chaining

In [ ]:
duck_df.select(
    """
                Date AS date, station_name, 
                Temp AS temp, "PM2.5" AS pm2_5,
            """
).filter(
    "Temp BETWEEN 10 AND 12AND Date BETWEEN DATE '2021-01-01' AND DATE '2021-01-31'"
).order("Temp").limit(5)

┌─────────────────────┬──────────────────────────────┬────────────────────┬───────────────────┐
│        date         │         station_name         │        temp        │       pm2_5       │
│      timestamp      │           varchar            │       double       │      double       │
├─────────────────────┼──────────────────────────────┼────────────────────┼───────────────────┤
│ 2021-01-08 00:00:00 │ PANACEA_001                  │ 10.000026105949011 │              34.0 │
│ 2021-01-25 23:00:00 │ PANACEA_000                  │ 10.000708271896553 │            8.1875 │
│ 2021-01-26 21:00:00 │ Athens CAMS station 1        │ 10.000905261684789 │         2.1124547 │
│ 2021-01-25 23:00:00 │ CleanAir in Greece - Thiseio │ 10.000995675604765 │ 8.657692307692308 │
│ 2021-01-29 19:00:00 │ PARASKEVI                    │ 10.001125804259836 │               8.0 │
└─────────────────────┴──────────────────────────────┴────────────────────┴───────────────────┘

## 3.4 Export Data

In [20]:
duck_sql = duckdb.sql(
    """
        SELECT 
            Date AS date,
            station_name, 
            Temp AS temp, 
            "PM2.5" AS pm2_5,
        FROM duck_df 
        WHERE 
            Temp BETWEEN 10 AND 12
            AND Date BETWEEN DATE '2021-01-01' AND DATE '2021-01-31'
        ORDER BY Temp
        LIMIT 5
    """
)

#### DuckDB Native Function

In [ ]:
duckdb.sql("COPY duck_sql TO 'output/temp_data_db.csv' (HEADER, DELIMITER ',');")

#### DuckDB to Pandas DataFrame

In [23]:
duck_sql.df().to_csv("output/temp_data_pd.csv", sep=",")

# 2. Why DuckDB is so Fast?

### 2.3 Parallel Execution

In [10]:
# Default my PC: 16 Threads - 10 Cores
duckdb.sql("SET threads = 16")
duckdb.sql("SELECT current_setting('threads')")

┌────────────────────────────┐
│ current_setting('threads') │
│           int64            │
├────────────────────────────┤
│                         16 │
└────────────────────────────┘

In [12]:
%%timeit
duckdb.sql(
    """
        SELECT 
            *
        FROM duck_df
    """
).df()

782 ms ± 8.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
duckdb.sql("SET threads = 1")
duckdb.sql("SELECT current_setting('threads')")

┌────────────────────────────┐
│ current_setting('threads') │
│           int64            │
├────────────────────────────┤
│                          1 │
└────────────────────────────┘

In [14]:
%%timeit
duckdb.sql(
    """
        SELECT 
            *
        FROM duck_df
    """
).df()

1.97 s ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2.4 Late Materialization

In [ ]:
print(
    duckdb.sql(
        """
        SELECT
            station_name, 
            Temp AS temp, 
            "PM2.5" AS pm2_5,
        FROM duck_df 
        WHERE 
            Date BETWEEN DATE '2021-01-01' AND DATE '2021-01-31'
        ORDER BY Temp
        LIMIT 5
    """
    ).explain("analyze")
)

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││         Total Time: 0.237s        ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌───────────────────────────┐
│      RESULT_COLLECTOR     │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│      EXPLAIN_ANALYZE      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│           TOP_N           │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           Top 5           │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│     duck_df."Temp" ASC    │
│

In [17]:
duck_parquet = duckdb.read_parquet("output.parquet")

In [ ]:
print(
    duckdb.sql(
        """
        SELECT
            station_name, 
            Temp AS temp, 
            "PM2.5" AS pm2_5,
        FROM duck_parquet 
        WHERE 
            Date BETWEEN DATE '2021-01-01' AND DATE '2021-01-31'
        ORDER BY Temp
        LIMIT 5
    """
    ).explain("analyze")
)

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││        Total Time: 0.0097s        ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌───────────────────────────┐
│      RESULT_COLLECTOR     │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│      EXPLAIN_ANALYZE      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│           TOP_N           │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           Top 5           │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│  duck_parquet."Temp" ASC  │
│